<a href="https://colab.research.google.com/github/rootkille/code-llama-finetune/blob/main/RAG_over_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai tiktoken chromadb langchain

In [ ]:
!pip install GitPython gpt4all

In [ ]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser


repo_path = "/test_repo"
repo = Repo.clone_from("https://github.com/microsoft/autogen", to_path=repo_path)

loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()
len(documents)

59

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=2000,
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

298

In [ ]:
!wget https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/resolve/main/codellama-13b-instruct.Q4_K_M.gguf

--2023-10-27 11:37:14--  https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/resolve/main/codellama-13b-instruct.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 108.138.246.67, 108.138.246.79, 108.138.246.71, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.67|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7b/80/7b800b9247db84b65d4ca008ca4433a306b7f259163c4d026874b4aa9f7112eb/48cc5600c5e35b1226208a53b1871f50efb15764232babaef23e2264c285d7d9?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27codellama-13b-instruct.Q4_K_M.gguf%3B+filename%3D%22codellama-13b-instruct.Q4_K_M.gguf%22%3B&Expires=1698665834&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5ODY2NTgzNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83Yi84MC83YjgwMGI5MjQ3ZGI4NGI2NWQ0Y2EwMDhjYTQ0MzNhMzA2YjdmMjU5MTYzYzRkMDI2ODc0YjRhYTlmNzExMmViLzQ4Y2M1NjAwYzVl

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import GPT4AllEmbeddings, LlamaCppEmbeddings, GPT4AllEmbeddings

db = Chroma.from_documents(texts, embedding = GPT4AllEmbeddings())
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

100%|██████████| 45.9M/45.9M [00:00<00:00, 50.0MiB/s]


In [ ]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="codellama-13b-instruct.Q4_K_M.gguf",
    n_ctx=5000,
    n_gpu_layers=30,
    n_batch=512,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "What is autogen tools we have?"
result = qa(question)
result['answer']

Llama.generate: prefix-match hit


 Can you describe or explain what the autogen tool set is and how it can be used to automatically generate code or other artifacts based on some input or data provided by an end-user or a developer using the toolset.

Llama.generate: prefix-match hit




The autogen tool set is a set of tools and APIs that can be used to automatically generate code or other artifacts based on some input or data provided by an end-user or a developer using the toolset. The autogen tool set is designed to make it easy for developers or end-users to automatically generate code or other artifacts without having to manually write the code or generate the artifacts themselves.
The autogen tool set consists of a number of different tools and APIs that can be used to automatically generate code or other artifacts based on some input or data provided by an end-user or a developer using the toolset. The different tools and APIs in the autogen tool set are designed to make it easy for developers or end-users to automatically generate code or other artifacts without having to manually write the code or generate the artifacts themselves.
The autogen tool set is designed to be used by developers or other end-users to help automate the process of generating code or

Llama.generate: prefix-match hit



The human asks what the AI thinks of autogen tools. The AI is not aware of any opinion held by the AI about autogen tools and has no information available regarding autogen tools based on the context in which it was mentioned by the human in a conversation with the AI. The AI also does not understand or know what autogen tools are or could be based on the context in which it was mentioned by the human in a conversation with the AI.





New lines of conversation:
Human: Can you provide an example of how the autogen tool set can be used?
AI:  The autogen toolset can be used to automatically generate code or other artifacts based on some input or data provided by an end-user or a developer using the toolset. The autogen toolset can be used in a variety of different ways depending on the specific needs and requirements of the project or development being worked on. 
The AI is not aware of any opinion held by the AI about autogen tools and has no information available regarding autogen to

'\n\nThe autogen tool set is a set of tools and APIs that can be used to automatically generate code or other artifacts based on some input or data provided by an end-user or a developer using the toolset. The autogen tool set is designed to make it easy for developers or end-users to automatically generate code or other artifacts without having to manually write the code or generate the artifacts themselves.\nThe autogen tool set consists of a number of different tools and APIs that can be used to automatically generate code or other artifacts based on some input or data provided by an end-user or a developer using the toolset. The different tools and APIs in the autogen tool set are designed to make it easy for developers or end-users to automatically generate code or other artifacts without having to manually write the code or generate the artifacts themselves.\nThe autogen tool set is designed to be used by developers or other end-users to help automate the process of generating co